In [5]:
import os
import math
import csv

import numpy as np
import geopandas as gpd

import rasterio
from rasterio.plot import show, plotting_extent
from rasterio.mask import mask
from rasterio.merge import merge

from grass_session import Session
import grass.script as gs
import grass.script.setup as gsetup

In [6]:
os.chdir('/home/shirobakaidou/docker_lab/morph_index/test/')
os.getcwd()

os.mkdir('output_test')

In [7]:
# Define Input Path
inpath = os.path.join(os.getcwd(), 'input')

# Define Output Path
outpath = os.path.join(os.getcwd(), 'output')

# Input DEM
dem_input = 'hydroDEM.tif'

# Input Basin
basin_input = 'basin.geojson'

# Define target CRS: WGS 84 / UTM zone 48N
dst_crs = 'EPSG:32648'

# Define GRASS Database
grass_dbase = os.path.join(os.getcwd(), 'grass')

# Define GRASS Location
grass_location = 'mylocation'

## Preprocessing

### 1. Reproject DEM

In [8]:
# Define function to reproject Raster
def reproject_raster(in_path, out_path, dst_crs):
    
    with rasterio.open(in_path) as src:
        transform, width, height = rasterio.warp.calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
    
        with rasterio.open(out_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                rasterio.warp.reproject(
                    source = rasterio.band(src, i),
                    destination = rasterio.band(dst, i),
                    src_transform = src.transform,
                    src_crs = src.crs,
                    dst_transform = transform,
                    dst_crs = dst_crs,
                    resampling = rasterio.warp.Resampling.nearest)
    return(print("The reprojected DEM is stored as: ", path_dem_prj))

In [9]:
# Input DEM
path_dem = os.path.join(inpath, dem_input)

# Output DEM
path_dem_prj = os.path.join(outpath, dem_input.split(".")[0]+"_prj."+dem_input.split(".")[1])

# Reproject
reproject_raster(in_path = path_dem, 
                 out_path = path_dem_prj, 
                 dst_crs = dst_crs)

The reprojected DEM is stored as:  /home/shirobakaidou/docker_lab/morph_index/test/output/hydroDEM_prj.tif


### 2. Reproject Basin

In [10]:
# Input Basin
path_basin = os.path.join(inpath, basin_input)

basin = gpd.read_file(path_basin)

# Reproject
basin_prj = basin.to_crs(dst_crs)

# Output Basin
path_basin_prj = os.path.join(outpath, basin_input.split(".")[0]+"_prj")
basin_prj.to_file(path_basin_prj)

### 3. Set up GRASS Environment

In [11]:
# Define GRASS Database
dbase = grass_dbase
location = grass_location

# Set GISBASE Environment Variable
gisbase = '/usr/lib/grass78'
os.environ['GISBASE'] = str(gisbase)

# Initialize
gsetup.init(os.environ['GISBASE'], dbase, location, 'PERMANENT')

print(gs.gisenv())

{'GISDBASE': '/home/shirobakaidou/docker_lab/morph_index/test/grass', 'LOCATION_NAME': 'mylocation', 'MAPSET': 'PERMANENT'}


In [12]:
# Create Init Location
gs.create_location(dbase, location)

In [13]:
# Create New Location in Target Projection
gs.run_command('g.proj', 
               flags = "c",
               georef = path_dem_prj)

Default region was updated to the new projection, but if you have multiple
mapsets `g.region -d` should be run in each to update the region from the
default
Projection information updated


0

In [14]:
# Switch to Projected Location, Mapset "PERMANENT"
gs.run_command('g.mapset', 
               location = grass_location,
               mapset = "PERMANENT")

0

In [15]:
# Install hydrology related GRASS Addons

#gs.run_command('g.extension', extension='r.basin', operation='add')
#gs.run_command('g.extension', extension='r.hypso', operation='add')
#gs.run_command('g.extension', extension='r.stream.basins', operation='add')
#gs.run_command('g.extension', extension='r.stream.distance', operation='add')
#gs.run_command('g.extension', extension='r.stream.order', operation='add')
#gs.run_command('g.extension', extension='r.stream.snap', operation='add')
#gs.run_command('g.extension', extension='r.stream.stats', operation='add')
#gs.run_command('g.extension', extension='r.width.funct', operation='add')
#gs.run_command('g.extension', extension='r.stream.channel', operation='add')
#gs.run_command('g.extension', extension='r.stream.segment', operation='add')
#gs.run_command('g.extension', extension='r.stream.slope', operation='add')

### 4. Mask DEM by Basin

***!!!NOTE: At this step, loop over each polygon in the SHP file.***

In [16]:
# Convert Geometry of the SHP into JSON Format, 
# so that rasterio can read it
def getFeatures(gdf):
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

basin_geom = getFeatures(basin_prj)

In [17]:
# Load Projected DEM
dem_prj = rasterio.open(path_dem_prj)

In [18]:
# Mask DEM by Basin
dem_msk, out_transform = mask(dataset=dem_prj, shapes=basin_geom, crop=True)

# Mask out NA
dem_msk = np.ma.masked_array(dem_msk, mask=(dem_msk <= 0))

### 5. Export Masked DEM

In [19]:
# Copy Metadata of projected DEM before masking
out_meta = dem_prj.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "count": dem_msk.shape[0],
    "height": dem_msk.shape[1],
    "width": dem_msk.shape[2],
    "transform": out_transform,
    "crs": dem_prj.crs
})

In [20]:
# Export Masked DEM
path_dem_msk = os.path.join(outpath, dem_input.split(".")[0]+"_msk."+dem_input.split(".")[1])
with rasterio.open(path_dem_msk, "w", **out_meta) as dest:
    dest.write(dem_msk)

## Calculate Hydromorph Param using GRASS

In [17]:
# Read DEM
gs.run_command('r.in.gdal', 
               flags='e', 
               input = path_dem_msk,
               output='r_elevation',
               quiet=True,
               overwrite=True)

0

In [18]:
## Set Region
gs.run_command('g.region', 
               flags="a",
               raster='r_elevation',
               quiet=True,
               overwrite=True)

0

In [19]:
# Define Threshold:
# source: https://github.com/OSGeo/grass-addons/blob/master/grass7/raster/r.basin/r.basin.py
resolution = gs.region()['nsres']
th = 1000000 / (resolution**2)
gs.message( "threshold : %s" % th )

threshold : 129.45662942175616


### 1. Watershed (Flow Direction & Flow Accumulation)

In [20]:
# Watershed SFD (single flow direction)
gs.run_command('r.watershed',
               flags='ams',
               elevation='r_elevation',
               accumulation='r_accumulation', # output accumulation
               drainage='r_drainage', # output flow direction
               convergence=5,
               quiet=True,
               overwrite=True)
gs.message("Flow Direction map and Flow Accumulation map done.")

Flow Direction map and Flow Accumulation map done.


### 2. Stream & Stream Order

In [21]:
# Stream Extraction
gs.run_command('r.stream.extract', 
               elevation = 'r_elevation',
               accumulation = 'r_accumulation',
               threshold = th,
               #d8cut =  1000000000,
               mexp = 0,
               stream_rast = 'r_stream_e', # output unique stream ids
               direction = 'r_drainage_e',
               quiet=True,
               overwrite=True)
gs.message("Stream Extraction done.")

Stream Extraction done.


In [22]:
# Create stream order maps: strahler, horton, hack, shreeve
gs.run_command('r.stream.order',
               stream_rast = 'r_stream_e',
               direction = 'r_drainage_e',
               strahler = 'r_strahler', # output Strahler order
               shreve = 'r_shreve', # output Shreve order
               horton = 'r_horton', # output Horton order
               hack = 'r_hack', # output Hack order
               quiet=True,
               overwrite=True)
gs.message("Stream order maps of Strahler, Horton, Hack and Shreeve are created.")

Stream order maps of Strahler, Horton, Hack and Shreeve are created.


### 3. Delineate Basin

In [23]:
# Delineation of basin: Create outlet
# Create outlet
gs.write_command('v.in.ascii',
                 input='-',
                 output='v_outlet',
                 sep=",",
                 stdin="%s,9999" % ("490282.15625,1966838"), # <modify coordinates>
                 quiet=True,
                 overwrite=True)
# The point is snapped to the nearest point which lies on the streamline
gs.run_command('r.stream.snap',
               input='v_outlet', 
               stream_rast='r_stream_e', # Input stream network
               output='v_outlet_snap', # output vector points map
               quiet=True,
               overwrite=True)
gs.run_command('v.to.rast',
               input='v_outlet_snap',
               output='r_outlet',
               use='cat',
               type='point',
               layer=1,
               value=1,
               quiet=True,
               overwrite=True)
# Delineates basins
gs.run_command('r.stream.basins',
               direction='r_drainage_e', # Input flow direction
               points='v_outlet_snap', # Input vector points
               basins='r_basin', # Output basin
               quiet=True,
               overwrite=True)
# Create Basin Mask (Vector)
gs.run_command('r.to.vect',
               input='r_basin',
               output='v_basin',
               type='area',
               flags='sv',
               quiet=True,
               overwrite=True)
gs.message("Delineation of basin done!")

Delineation of basin done!


### P: Basin Area & Perimeter

In [24]:
# Add two columns to the table 'basin': Area and Perimeter
gs.run_command('v.db.addcolumn',
               map='v_basin',
               columns='area double precision') # name & type
gs.run_command('v.db.addcolumn',
               map='v_basin',
               columns='perimeter double precision')

# Populate Perimeter Column
gs.run_command('v.to.db',
               map='v_basin',
               type='line,boundary',
               layer=1,
               qlayer=1,
               option='perimeter',
               units='kilometers',
               columns='perimeter',
               quiet=True,
               overwrite=True)
gs.run_command('v.to.db',
               map='v_basin',
               type='line,boundary',
               layer=1,
               qlayer=1,
               option='area',
               units='kilometers',
               columns='area',
               #flags='c',
               quiet=True,
               overwrite=True)
gs.message("Column 'Area' and 'Perimeter' are populated!")

Column 'Area' and 'Perimeter' are populated!


In [25]:
# Export Selected Attributes
gs.run_command('v.db.select',
               map='v_basin',
               column='perimeter, area',
               file=os.path.join(outpath,'basin_area_perimeter'),
               overwrite=True)

# Read Perimeter and Area from the text file
tmp = open(os.path.join(outpath,"basin_area_perimeter"), "r")
tmp = tmp.read()

# Extract Perimeter and Area
basin_perimeter = float(tmp.split('\n')[1].split('|')[0])
basin_area = float(tmp.split('\n')[1].split('|')[1])
print("The perimeter of the basin is",basin_perimeter, "km; \n",
      "The area of the basin is", basin_area, "km2") # both km

The perimeter of the basin is 111.389201 km; 
 The area of the basin is 263.591175 km2


### P: Circularity Ratio

In [26]:
# Circularity Ratio
circularity_ratio = (4*math.pi*basin_area)/(basin_perimeter**2)
print("Circularity Ratio is", circularity_ratio)

Circularity Ratio is 0.26696513826844187


### 4. Outlet

In [27]:
# Get coordinates of outlet (belonging to stream network)
gs.run_command('v.db.addtable', map='v_outlet_snap',
               quiet=True,
               overwrite=True)

gs.run_command('v.db.addcolumn',
               map='v_outlet_snap',
               columns='x double precision, y double precision',
               quiet=True,
               overwrite=True)

gs.run_command('v.to.db',
               map='v_outlet_snap',
               option='coor',
               col='x,y',
               quiet=True,
               overwrite=True)

gs.run_command('v.out.ascii',
               input='v_outlet_snap',
               output=os.path.join(outpath,'outlet_coords.txt'),
               cats=1,
               format="point",
               quiet=True,
               overwrite=True)

outlet_coords = open(os.path.join(outpath,'outlet_coords.txt')).readline()
east_o, north_o, cat = outlet_coords.split('|')
print("The east and north coordinateof outlet is", east_o, ",", north_o)

The east and north coordinateof outlet is 490282.15625 , 1966838


### 5. Mainchannel

In [28]:
# Main Channel
gs.mapcalc ("$r_mainchannel = if($r_hack==1,1,null())",
            r_hack = 'r_hack', # input
            r_mainchannel = 'r_mainchannel', # output
            quiet=True,
            overwrite=True) 

gs.run_command("r.thin", 
               input = 'r_mainchannel',
               output = 'r_mainchannel'+'_thin',
               quiet=True,
               overwrite=True)

gs.run_command('r.to.vect',
               input='r_mainchannel'+'_thin',
               output='v_mainchannel',
               type='line',
               #verbose=True,
               quiet=True,
               overwrite=True)
gs.message("Main Channel is extracted.")

Main Channel is extracted.


### P: Main Channel Length (!= Basin Length)

In [129]:
# Basin Length
param_mainchannel = gs.read_command('v.what',
                                    map = 'v_mainchannel',
                                    coordinates='%s,%s' % (east_o, north_o),
                                    distance=5)
MCL = float(param_mainchannel.split('\n')[7].split()[1])/1000 # km

print("The Main Channel Length is", MCL, "km.")

The Main Channel Length is 75.032720702 km.


### P: Elongation Ratio

In [30]:
# Elongation Ratio
elon_ratio = (2*math.sqrt(basin_area/math.pi))/MCL

print("The elongation ratio is ", elon_ratio)

The elongation ratio is  0.24415734740038567


### P: Form Factor

In [31]:
# Form Factor
form_factor = basin_area/(MCL**2)
print("The Form Factor is ", form_factor)

The Form Factor is  0.046819791716406545


### P: Relative Relief

In [32]:
# Relative Relief (Hmax - Hmin)
height_attr = gs.read_command('r.info', 
                flags='r',
                map='r_elevation')
height_min = float(height_attr.strip().split('\n')[0].split('=')[-1])
height_max = float(height_attr.strip().split('\n')[1].split('=')[-1])
print("The max Altitude is ", height_min, "m; \n", 
      "The min Altitude is ", height_max, "m.")

relative_relief = height_max - height_min
print("The relative Relief is ", relative_relief, "m.") # meter

The max Altitude is  165.0 m; 
 The min Altitude is  1532.0 m.
The relative Relief is  1367.0 m.


### P: Relief Ratio

In [34]:
# Relief Ratio
relief_ratio = (relative_relief/1000)/MCL
print("The Relief Ratio is ", relief_ratio)

The Relief Ratio is  0.018218718276645972


### P: Dissection Index

In [35]:
# Dissection Index
dissection_index = relative_relief/height_max
print("The Dissection Index is ", dissection_index)

The Dissection Index is  0.8922976501305483


### P: Hypsometric Integral

In [36]:
# Mean Elevation
gs.run_command("r.stats.zonal",
               base = 'r_basin',
               cover = 'r_elevation',
               method = 'average',
               output = 'r_height_average',
               quiet = True,
               overwrite = True)
mean_elev = float(gs.read_command('r.info',
                                  flags = 'r',
                                  map = 'r_height_average').split('\n')[0].split('=')[1])
print("Mean Elevation is ", mean_elev, "m")

Mean Elevation is  455.9324 m


In [37]:
# Hypsometric Integral
hypsom = (mean_elev-height_min)/relative_relief
print("Hypsometric Integral is ", hypsom)

Hypsometric Integral is  0.2128254572055596


### 6. Slope & Aspect

In [38]:
# Creation of Slope and Aspect maps
gs.run_command('r.slope.aspect',
               elevation='r_elevation',
               slope='r_slope',
               aspect = 'r_aspect',
               quiet=True,
               overwrite=True)
gs.message("Slope and Aspect maps done.")

Slope and Aspect maps done.


### P: Average Basin Slope

In [39]:
slope_baricenter = gs.read_command("r.volume", 
                                   input = 'r_slope',
                                   clump = 'r_basin',
                                   quiet = True).split()

basin_slope = float(slope_baricenter[30])
print("Average Basin Slope is ", basin_slope)

Average Basin Slope is  11.38


### 7. Topological Diameter

In [40]:
gs.mapcalc("$r_mainchannel_dim = -($r_mainchannel - $r_shreve) + 1",
           r_mainchannel_dim = 'r_mainchannel_dim',
           r_shreve = 'r_shreve',
           r_mainchannel = 'r_mainchannel',
           quiet = True,
           overwrite = True)

gs.run_command('r.thin',
               input = 'r_mainchannel_dim',
               output = 'r_mainchannel_dim_thin',
               quiet = True,
               overwrite = True)

gs.run_command('r.to.vect',
               input = 'r_mainchannel_dim_thin',
               output = 'v_mainchannel_dim',
               type = 'line',
               flags = 'v',
               quiet = True,
               overwrite = True)

         overwritten


0

### P: Average Mainchannel Slope

In [41]:
gs.run_command('v.to.points',
               input = 'v_mainchannel_dim',
               output = 'v_mainchannel_dim_point',
               type = 'line',
               quiet = True,
               overwrite = True)

vertex = gs.read_command('v.out.ascii',
                         input = 'v_mainchannel_dim_point',
                         quiet = True,
                         overwrite = True).strip().split('\n')

nodi = np.zeros((len(vertex), 4), float)
pendenze = []

for i in range(len(vertex)):
    x, y = float(vertex[i].split('|')[0]), float(vertex[i].split('|')[1])
    vertice1 = gs.read_command('r.what',
                               map = 'r_elevation',
                               coordinates = '%s,%s' % (x,y))
    vertice = vertice1.replace('\n', '').replace('||', '|').split('|')
    nodi[i, 0], nodi[i, 1], nodi[i, 2] = float(vertice[0]), float(vertice[1]), float(vertice[2])
    
for i in range(0, len(vertex)-1, 2):
    dist = math.sqrt(math.fabs((nodi[i, 0] - nodi[i+1, 0]))**2 + math.fabs((nodi[i, 1] - nodi[i+1, 1]))**2)
    deltaz = math.fabs(nodi[i, 2] - nodi[i+1, 2])
    
    # Control to prevent float division by zero (dist=0)
    try:
        pendenza = deltaz / dist
        pendenze.append(pendenza)
        mainchannel_slope = float(sum(pendenze) / len(pendenze)*100)
    except:
        pass

print("Mean slope of mainchannel is", mainchannel_slope, "%")

         overwritten


Mean slope of mainchannel is 1.8217512567769045 %


### P: Channel Gradient

In [42]:
stream_stats = gs.read_command('r.stream.stats',
                               stream_rast = 'r_strahler', # NOT Horton???
                               direction = 'r_drainage_e',
                               elevation = 'r_elevation',
                               quiet = True)

stream_stats_summary = stream_stats.split('\n')[4].split('|')
stream_stats_mom = stream_stats.split('\n')[8].split('|')

Len_streams = float(stream_stats_summary[2])
Bif_ratio = float(stream_stats_mom[0])

print("Total stream length is ", Len_streams, "km \n",
      "Bifurcation Ratio is ", Bif_ratio)

Total stream length is  711.1799 km 
 Bifurcation Ratio is  3.4345


In [43]:
channel_gradient = relative_relief / ((math.pi/2) * ((Len_streams/(Len_streams-1)) / Bif_ratio))

print("Channel Gradient is", channel_gradient)

Channel Gradient is 2984.702579845911


### P: Slope Ratio

In [44]:
slope_ratio = mainchannel_slope / basin_slope

Slope_ratio = float(stream_stats_mom[3])

print("Slope Ratio is ", slope_ratio, "or", Slope_ratio)

Slope Ratio is  0.1600835902264415 or 3.293


## Write Indices as Attributes into Basin Shapefile

In [124]:
basin_prj[0:]

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,geometry
0,4050060240,4051108500,4050060240,4050017020,0.0,917.6,1227.3,1227.3,44270,2,0,2,710,"MULTIPOLYGON (((487622.092 1946396.778, 486480..."


In [138]:
basin_pop = basin_prj[0:].assign(perimeter_km = basin_perimeter,
                                 area_km = basin_area,
                                 outlet_east = east_o,
                                 outlet_north = north_o,
                                 
                                 elevation_max_m = height_max,
                                 elevation_min_m = height_min,
                                 relative_relief_m = relative_relief,
                                 elevation_mean_m = mean_elev,
                                 
                                 mainchannel_length_km = MCL,
                                 total_stream_length_km = Len_streams,
                                 
                                 avg_basin_slope = basin_slope,
                                 avg_mainchannel_slope_percent = mainchannel_slope,
                                 
                                 circularity_ratio = circularity_ratio,
                                 elongation_ratio = elon_ratio,
                                 form_factor = form_factor,
                                 relief_ratio = relief_ratio,
                                 dissection_index = dissection_index,
                                 hypsometric_integral = hypsom,
                                 bifurcation_ratio = Bif_ratio,
                                 channel_gradient = channel_gradient,
                                 slope_ratio = slope_ratio
                                 )

In [139]:
basin_pop[0:]

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,geometry,perimeter_km,area_km,outlet_east,outlet_north,elevation_max_m,elevation_min_m,relative_relief_m,elevation_mean_m,mainchannel_length_km,total_stream_length_km,avg_basin_slope,avg_mainchannel_slope_percent,circularity_ratio,elongation_ratio,form_factor,relief_ratio,dissection_index,hypsometric_integral,bifurcation_ratio,channel_gradient,slope_ratio
0,4050060240,4051108500,4050060240,4050017020,0.0,917.6,1227.3,1227.3,44270,2,0,2,710,"MULTIPOLYGON (((487622.092 1946396.778, 486480...",111.389201,263.591175,490282.15625,1966838,1532.0,165.0,1367.0,455.9324,75.032721,711.1799,11.38,1.821751,0.266965,0.244157,0.04682,0.018219,0.892298,0.212825,3.4345,2984.70258,0.160084


In [140]:
# Output
path_basin_index = os.path.join(outpath, basin_input.split(".")[0]+"_index")
basin_pop.to_file(path_basin_index)